In [1]:
import networkx as nx
import pickhardtpayments as pp
import random

In [2]:
graph = pp.ChannelGraph("channels.sample.json")

In [3]:
class SimulatedOracle(pp.OracleLightningNetwork):
    def __init__(self,channel_graph):
        super().__init__(channel_graph)
    
    def conduct_payment(self,src,dest,amt):
        if self.network.has_edge(dest, src):
            items = self.network[src][dest]
            for short_channel_id, channel in items.items():
                channel = channel["channel"]
                if channel.can_forward(amt):
                    channel.actual_liquidity = channel.actual_liquidity - amt
                    other_channel= self.network[dest][src][short_channel_id]["channel"]
                    other_channel.actual_liquidity = other_channel.actual_liquidity + amt
                    return True
        return False

    def _next_hop(self, path):
        """                                                                                                 
        generator to iterate through edges indext by node id of paths                                       
                                                                                                            
        The path is a list of node ids. Each call returns a tuple src, dest of an edge in the path          
        """
        for i in range(1, len(path)):
            src = path[i-1]
            dest = path[i]
            yield (src, dest)

    
    def settle_payment(self, path, amt):
        channels = []
        for src, dest in self._next_hop(path):
            fwd = False
            if self.network.has_edge(dest, src):
                items = self.network[src][dest]
                for short_channel_id, channel in items.items():
                    channel = channel["channel"]
                    if channel.can_forward(amt):
                        fwd = True
                        channels.append(channel)
                        break
            if fwd==False:
                return False
        for channel in channels:
            channel.actual_liquidity = channel.actual_liquidity - amt
            other_channel= self.network[channel.dest][channel.src][channel.short_channel_id]["channel"]
            other_channel.actual_liquidity = other_channel.actual_liquidity + amt
        return True

In [4]:
s = SimulatedOracle(graph)
nodes = list(s.network.nodes())

In [5]:
def get_payment_pair(nodes):
    src = random.choice(nodes)
    dest = src
    while dest == src:
        dest = random.choice(nodes)
    return src, dest

def generate_payment_pairs(nodes, num_pairs =1000_000):
    res = []
    amts = list(i for i in range(1,100))
    for i in range(num_pairs):
        src,dest = get_payment_pair(nodes)
        amt = random.choice(amts)
        res.append((src,dest,amt))
    return res

def compute_total_flows(txs,nodes):
    flows = {n:0 for n in nodes}
    for src,dest, amt in txs:
        flows[src]-=amt
        flows[dest]+=amt
    return flows

In [6]:
txs = generate_payment_pairs(nodes)
print(compute_total_flows(txs,nodes))

{'A': 24399, 'B': -9113, 'C': 15348, 'D': -30634}


In [7]:
uncertainty_network = pp.UncertaintyNetwork(graph)

graph = nx.DiGraph()
for src,dest,key, channel in uncertainty_network.network.edges(data = "channel", keys=True):
    #channel = uncertainty_network.network[src][dest]["channel"]
    graph.add_edge(src,dest,channel = channel, cost = channel.linearized_integer_routing_unit_cost())
    #print(src,dest)


paths = {n: {} for n in nodes}
for n in nodes:
    for m in nodes:
        if n==m:
            continue
        #res = nx.dijkstra_path(graph, n,m,weight="cost")
        res = nx.all_simple_paths(graph,n,m,cutoff=5)
        paths[n][m]=[p for p in res]
        #print(res)
cnt = 0
print(cnt)
suc = 0
fail = 0
attempts = 0
fa = 0
for src,dest,amt in txs:
    #session.pickhardt_pay(src,dest,amt)
    #res = nx.dijkstra_path(graph, src,dest,weight="cost")
    failed = True
    for path in paths[src][dest]: 
        res = s.settle_payment(path, amt)
        attempts +=1
        if res == True:
            suc +=1
            failed = False
            break
        else:
            fa +=1
    if failed == True:
        fail+=1
    cnt +=1
    if cnt % 20_000 == 0:
        print("payments: {} failed_payment_rate: {} attempts: {} fa: {} failed_attempts_rate: {}".format(
            cnt, 
            float(fail)/cnt, 
            attempts,
            fa,
            float(attempts-cnt)/cnt))
        #break
    #print(a,b)
    #break
    #oracle.send_onion([src,dest],amt)

0
payments: 20000 failed_payment_rate: 0.0 attempts: 22360 fa: 2360 failed_attempts_rate: 0.118
payments: 40000 failed_payment_rate: 0.0 attempts: 48483 fa: 8483 failed_attempts_rate: 0.212075
payments: 60000 failed_payment_rate: 0.0 attempts: 75312 fa: 15312 failed_attempts_rate: 0.2552
payments: 80000 failed_payment_rate: 0.0 attempts: 101799 fa: 21799 failed_attempts_rate: 0.2724875
payments: 100000 failed_payment_rate: 0.0 attempts: 128249 fa: 28249 failed_attempts_rate: 0.28249
payments: 120000 failed_payment_rate: 0.0 attempts: 154563 fa: 34563 failed_attempts_rate: 0.288025
payments: 140000 failed_payment_rate: 0.0 attempts: 181393 fa: 41393 failed_attempts_rate: 0.2956642857142857
payments: 160000 failed_payment_rate: 0.0 attempts: 207998 fa: 47998 failed_attempts_rate: 0.2999875
payments: 180000 failed_payment_rate: 0.0 attempts: 234674 fa: 54674 failed_attempts_rate: 0.30374444444444443
payments: 200000 failed_payment_rate: 0.0 attempts: 260533 fa: 60533 failed_attempts_rate:

In [8]:
paths

{'A': {'B': [['A', 'B'], ['A', 'C', 'B'], ['A', 'D', 'B']],
  'C': [['A', 'B', 'C'], ['A', 'C'], ['A', 'D', 'B', 'C']],
  'D': [['A', 'B', 'D'], ['A', 'C', 'B', 'D'], ['A', 'D']]},
 'B': {'A': [['B', 'A'], ['B', 'C', 'A'], ['B', 'D', 'A']],
  'C': [['B', 'A', 'C'], ['B', 'C'], ['B', 'D', 'A', 'C']],
  'D': [['B', 'A', 'D'], ['B', 'C', 'A', 'D'], ['B', 'D']]},
 'C': {'A': [['C', 'B', 'A'], ['C', 'B', 'D', 'A'], ['C', 'A']],
  'B': [['C', 'B'], ['C', 'A', 'B'], ['C', 'A', 'D', 'B']],
  'D': [['C', 'B', 'A', 'D'],
   ['C', 'B', 'D'],
   ['C', 'A', 'B', 'D'],
   ['C', 'A', 'D']]},
 'D': {'A': [['D', 'A'], ['D', 'B', 'A'], ['D', 'B', 'C', 'A']],
  'B': [['D', 'A', 'B'], ['D', 'A', 'C', 'B'], ['D', 'B']],
  'C': [['D', 'A', 'B', 'C'],
   ['D', 'A', 'C'],
   ['D', 'B', 'A', 'C'],
   ['D', 'B', 'C']]}}

In [23]:
x = nx.all_simple_paths(graph,"A","B",cutoff=5)
for path in x:
    print(path)


['A', 'B']
['A', 'C', 'B']
['A', 'D', 'B']


# TODOS:

* account fees
* add strategies

In [13]:
0.002 * 1_000 * 3600*24 * 365 / 100_000_000


0.63072

In [14]:
3.9*12

46.8